Now we parse the content of the JSON file with the QASM circuits

In [1]:
using ITensors, ITensorMPS
using IterTools
using Plots
using Glob
using JSON

include("simulation/mpo_simulation.jl")
using .MPOSimulation
include("utils/utils.jl")
using .Utils
include("simulation/mps_simulation.jl")
using .MPSSimulation
include("simulation/reconstruct_expectation.jl")
using .ReconstructExpectation


In [18]:
# Simulation using MPS - single timestep
json_filename = "subcircuits/subcircuits.json"  
basis_gates = ["h", "rx", "ry", "rz", "rxx","rzz", "ryy", "cx"]
cutoff = 1e-12
maxdim = 50
method = "densitymatrix" #currently this is not used


circuits_data, expval_knitting, coefficients = parse_subcircuits(json_filename);
all_mpo_sequences = build_all_mpo_sequences(circuits_data, basis_gates);
all_psis = apply_mpo_sequences(circuits_data, all_mpo_sequences, cutoff, maxdim, method);
expval_tensors = reconstruct_expectation(all_psis, coefficients);

discrepancy = abs(round(100 * (expval_knitting - expval_tensors) / expval_knitting, digits=4))
println("""
==============================
• Expected Value Knitting: $(round(expval_knitting, digits=8))
• Expected Value Tensors:  $(round(expval_tensors, digits=8))
• Discrepancy            : $discrepancy%
==============================
""")


Completed applying MPO sequence for subcircuit A1.
• Expected Value Knitting: 4.48418344
• Expected Value Tensors:  4.48418487
• Discrepancy            : 0.0%



In [17]:
# Simulation using MPS - multiple timesteps
basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cx"]
cutoff = 1e-12
maxdim = 50
method = "densitymatrix" # Currently not used

# Directory containing the subcircuit JSON files
subcircuits_dir = "subcircuits/"

json_files = Glob.glob(joinpath(subcircuits_dir, "*.json"))

struct ExpValResult
    filename::String
    expval_knitting::Float64
    expval_tensors::Float64
    discrepancy::Float64
end

expval_results = ExpValResult[] 

for json_filename in json_files

    circuits_data, expval_knitting, coefficients = parse_subcircuits(json_filename)
    all_mpo_sequences = build_all_mpo_sequences(circuits_data, basis_gates)
    all_psis = apply_mpo_sequences(circuits_data, all_mpo_sequences, cutoff, maxdim, method)
    expval_tensors = reconstruct_expectation(all_psis, coefficients)
    
    discrepancy = abs(round(100 * (expval_knitting - expval_tensors) / expval_knitting, digits=4))
    
    println("""
    ==============================
    • File: $(basename(json_filename))
    • Expected Value Knitting: $(round(expval_knitting, digits=8))
    • Expected Value Tensors:  $(round(expval_tensors, digits=8))
    • Discrepancy            : $discrepancy%
    ==============================
    """)
    
    push!(expval_results, ExpValResult(basename(json_filename), expval_knitting, expval_tensors, discrepancy))
end



Completed applying MPO sequence for subcircuit A1.
• File: subcircuits.json
• Expected Value Knitting: 4.48418344
• Expected Value Tensors:  4.48418487
• Discrepancy            : 0.0%



<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
After this, MPO Simulation (deprecated soon)
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>

In [9]:
# Simulation using MPO
basis_gates = ["h", "rx", "ry", "rz", "rxx","rzz", "ryy", "cx"]
cutoff = 1e-12
maxdim = 10000
method = "densitymatrix" #currently this is not used

all_mpo_sequences, all_rhos = mpo_sequence_apply(circuits_data,basis_gates, cutoff, maxdim, method);

# Does not seem like a very efficient way, the old one is better

labels = keys(all_rhos)

# Collect site indices for each label (we take generic state from all_psis)
sites_per_label = Dict{String, Vector{Index{Int64}}}()
for label in labels
    sites_per_label[label] = Vector{Index{Int64}}()
    for inds in siteinds(all_rhos[label][2])
        push!(sites_per_label[label], inds[2])
    end
end


# Generate union of sites
sites_union = vcat(union(values(sites_per_label))...)

function create_mpo(op_name::String, site::Int, sites::Vector{Index{Int64}})
    os = OpSum()
    add!(os, op_name, site)
    return MPO(os, sites)
end

# Create the observable 
obs = sum([create_mpo("Z", i, sites_union) for i in 1:length(sites_union)]) # ZII + IZI + IIZ
# obs = [MPO(sites_AB, "Z")] # ZZZ

function mpo_tensor_product(mpo_dict::Dict{String, MPO}, sites::Vector{Index{Int64}})
    total_length = sum(length(mpo) for mpo in values(mpo_dict))
    rho = MPO(sites)
    current_index = 1
    for label in keys(mpo_dict)
        mpo = mpo_dict[label]
        for i in 1:length(mpo)
            rho[current_index] = mpo[i]
            current_index += 1
        end
    end
    return rho
end

# We reconstruct the expectation value (with MPO)
# Tr[Oρ'] = ∑ₓⁿ Tr[Oᴀₓ ⊗ Oᵦₓ ρ'] = ∑ₓⁿ ∑ₖ₌₁ᴷ ∑ⱼₐ₌₁ᵐₐ ∑ⱼᵦ₌₁ᵐᵦ αₖ pᴀⱼₐ pᵦⱼᵦ ⟨Oᴀₓ⟩ₖ,ⱼₐ ⟨Oᵦₓ⟩ₖ,ⱼᵦ
expval_tensors = 0.0

for (k, coeff) in enumerate(coefficients)
    temp = 0.0
    mpo_dict = Dict{String, MPO}()
    for label in labels
        mpo_dict[label] = all_rhos[label][k]
    end
    rho = mpo_tensor_product(mpo_dict, sites_union)
    expval_tensors += real(coeff * tr(apply(rho, obs)))
end

discrepancy = abs(round(100 * (expval_knitting - expval_tensors) / (expval_knitting), digits=4))

println("""
==============================
• Expected Value Knitting: $(round(expval_knitting, digits=8))
• Expected Value Tensors:  $(round(expval_tensors, digits=8))
• Discrepancy            : $discrepancy%
==============================
""")


• Expected Value Knitting: 12.96961105
• Expected Value Tensors:  12.96961111
• Discrepancy            : 0.0%

